In [44]:
import logging
import time
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
from distributed import LocalCluster, Client

import matplotlib
import hvplot.pandas

In [45]:
base_path = Path('../')

In [59]:
unsorted = pd.read_csv(base_path / 'compared_cases/unsorted/unsorted_results_df_2020-06-13 19:19:20.735300.csv', index_col=0)
sorted_geohash = pd.read_csv(base_path / 'compared_cases/sorted_geohash/2020-06-11 21:43:03.231475_unsorted_results_df.csv', index_col=0)
# spatially_sorted = pd.read_csv()

In [60]:
unsorted

,num_polygons,num_points,num_result_points,sort_time_sec,time_min,total_points,projected_total_time_hr
0,1,111187928,1031,0,44.221518,111187928,0.737025
1,10,111187928,6551,0,42.961035,111187928,0.716017
2,100,111187928,203079,0,42.700080,111187928,0.711668


In [69]:
# add sort/query times in seconds
# unsorted
unsorted['sort_time_sec'] = 0
unsorted['method'] = 'unsorted'
unsorted['query_time_sec'] = unsorted['time_min']*60
unsorted['preprocess_time_sec'] = 0
unsorted['total_time'] = unsorted.sort_time_sec + unsorted.preprocess_time_sec + unsorted.query_time_sec

# sorted_geohash
sorted_geohash['sort_time_sec'] = 0.08800150308344099 * 60 * 60
sorted_geohash['method'] = 'sorted_geohash'
sorted_geohash['query_time_sec'] = sorted_geohash['time_min']*60
sorted_geohash['preprocess_time_sec'] = 1 # need to fix this one, once I time the geohash creation
sorted_geohash['total_time'] = sorted_geohash.sort_time_sec + sorted_geohash.preprocess_time_sec + sorted_geohash.query_time_sec

# spatially_sorted


In [72]:
# drop unneeded columns

# concatenate results
results = pd.concat([unsorted, sorted_geohash], axis=0).reset_index(drop=True)

In [73]:
results

,num_polygons,num_points,num_result_points,sort_time_sec,time_min,total_points,projected_total_time_hr,method,query_time_sec,preprocess_time_sec,total_time
0,1,111187928,1031,0.000000,44.221518,111187928,0.737025,unsorted,2653.291054,0,2653.291054
1,10,111187928,6551,0.000000,42.961035,111187928,0.716017,unsorted,2577.662106,0,2577.662106
2,100,111187928,203079,0.000000,42.700080,111187928,0.711668,unsorted,2562.004811,0,2562.004811
3,1,111187928,1031,316.805411,0.028365,111187928,0.000473,sorted_geohash,1.701927,1,319.507338
4,10,111187928,6551,316.805411,0.141760,111187928,0.002363,sorted_geohash,8.505605,1,326.311016
5,100,111187928,203079,316.805411,1.615742,111187928,0.026929,sorted_geohash,96.944520,1,414.749931


In [74]:
results.hvplot.scatter(x='num_polygons', y='total_time', color='method', logx=False, logy=False, xlabel='Number of Polygons', ylabel='Total Time (s)')

:Scatter   [num_polygons]   (total_time,method)